# Processing Polish Data

In [1171]:
import pandas as pd
import datetime
import re
from collections import defaultdict
TODAY = datetime.date.today()
CURRENT_YEAR = TODAY.year
DEFAULT_LAT = "52.2337172" # points to Warsaw, Poland
DEFAULT_LON = "21.0714322"

## Finds Processor

Sometimes dates are written as XX' of XXth c.; converts these expressions to yyyy format

In [1172]:
def convertCentury(date):
    match = re.search(r"st|nd|th", date) # date is in century form
    if (match):
        half = re.split("of", date)

        subCenturyText = re.search(r"[0-9][0-9]|[0-9]", half[0])
        subCenturyInt = int(subCenturyText.group()) # this is the last 2 digits of yyyy

        centuryText = re.search(r"[0-9][0-9]|[0-9]", half[1])
        centuryInt = int(centuryText.group()) # the century, e.g. 20th century

        year = ((centuryInt - 1) * 100) + subCenturyInt
        start = subCenturyText.start()
        end = subCenturyText.end() + centuryText.end() + 8
        date = half[0] + half[1]
        date = date[:start] + str(year) + date[end:]
        return date

The function below parses for year found. Some variation in input causes complexity here; for example, here are some inputs:
<br>
before Oct.2010
<br>
bef. 1960
<br>
2005 or earlier
<br>
around 1930
<br>
06 and 11.2006
<br>
1.11.1865
<br>
c. 1900-1914
<br>
1880s
<br>
21 June 1875
<br>
Spring 1962
<br>
1926/27
<br>
1872 or 1875
<br>
Therefore, new cases may need to be covered if new variations arise.

In [1173]:
def get_date(find: dict, date):
    if (re.fullmatch(r"([0-9][0-9][0-9][0-9])+$", date)):  # string is simple, only 4 consecutive digits, e.g. '2003'
        find["year_found"] = date
        find["year_found_end"] = date
        return
# at this point, date can be complex. First make sure date is not in century format
    if (re.search(r"[0-9](st|nd|th)", date)):
        date = convertCentury(date) # if century format, convert it to yyyy
        
    year = re.search(r"([0-9][0-9][0-9][0-9])", date).group() # extracted year string
    
    if (re.search(r"bef|earl|pre", date)): # if end of range specified
        find["year_found"] = "1700"
        find["year_found_end"] = year
        return
    
    if (re.search(r"after|later|post", date)): # if start of range specified
        find["year_found"] = year
        find["year_found_end"] = CURRENT_YEAR
        return
    
    # if full range is specified (checking for 4 digits to eliminate '-' being used between days: 26-27 June, 2003)
    if (re.search(r"[1-2][0-9][0-9][0-9]\s*(or|/|-|to|and)", date) and not re.search(r"ctober", date)):
        year = re.findall(r"([0-9][0-9][0-9][0-9])", date)
        find["year_found"] = year[0]
        if (year.__len__() != 1): # if string looks like 1991-2003
            find["year_found_end"] = year[1]
            return
        else: # string looks like 1961-64
            pattern = r"or|/|-|to|and"
            endYear = year[0][:2] + re.search(r"[0-9][0-9]", re.split(pattern, date, 1)[1]).group()  # construct full end year: 64 -> 1964
            if (int(endYear) < int(year[0])): endYear = str(int(endYear) + 100) # in case of 1997-01
            find["year_found_end"] = endYear
            return

    if (re.search(r"[1-2][0-9][0-9]0s", date)): # if decade range given, e.g. 1880s
        year = re.search(r"[1-2][0-9][0-9]", date).group()
        find["year_found"] = year + "0"
        find["year_found_end"] = year + "9"
        return
    
    # at this point no special markers (before, after, -, etc.) left, just take the year and ignore other characters
    find["year_found"] = year
    find["year_found_end"] = year

Determines if findSpotName contains a digit or Roman numeral

In [1174]:
def containsNumber(place) -> bool:
    pattern = re.compile(r"[0-9]|(\s(I|V|X|L|C)(?=(\s|$|I|V|X|L|C)))")
    if (re.search(pattern, str(place))): 
        return True
    else: return False

Determines what finds are single finds, hoards, and excavations

In [1175]:
def singleOrHoard(freq, finds): 
    for find in finds:
        place = find["Name"]
        year = find["tempYear"]
        if (len(freq[(place, year)]) > 1 and containsNumber(place)): # then it's part of a hoard.
            find["type_find"] = "Hoard"
            find["num_coins"] = len(freq[(place, year)])
            find["hoard?"] = True
            find["single?"] = False
            duplicates = freq[(place, year)]
            index = 1 # to keep the 0th one from being removed
            representativeEntry = duplicates[0] # the 0th one
            while (index < len(freq[(place, year)])):
                duplicate = duplicates[index]

                if (int(duplicate["find_start_date"] < int(representativeEntry["find_start_date"]))):
                    representativeEntry["find_start_date"] = duplicate["find_start_date"]
                
                if (int(duplicate["find_end_date"]) > int(representativeEntry["find_end_date"])):
                    representativeEntry["find_end_date"] = duplicate["find_end_date"]
                
                groupToFind[duplicate["FindNumber"]] = representativeEntry["FindNumber"] # to later map groups to the correct find id.
                finds.remove(duplicate)
                index += 1
        else: 
            find["type_find"] = "Single Find"
            find["num_coins"] = 1
            find["hoard?"] = False
            find["single?"] = True
        find["excavation?"] = bool(re.search(r"excav", str(find["tempCircumstances"]), re.IGNORECASE)) # check for "excav" in findCircumstance
        find.pop("tempYear") # don't need these anymore
        find.pop("tempCircumstances")

Setting up dataframe as a list of dicts

In [1176]:
finds = list()
freq = defaultdict(list)
singleFinds = defaultdict(list)
find_id_to_Index = {}
years = {}
path = "C:/Users/there/Documents/Code/Numismatics/AFE PL 325-750 (Polish Data).xlsx"
theirData = pd.read_excel(path)
for index, row in theirData.iterrows():
    find = {}
    find["FindNumber"] = row["ID"]
    find["Name"] = row["FindspotName"]
    place = find["Name"]
    year = row["YearFound"]

    freq[(place, year)].append(find)

    find["lat"] = row["LatitudePlace"]
    find["long"] = row["LongitudePlace"]
    find["cf_custom_region_vague"] = 2
    if (pd.isnull(find["lat"]) or pd.isnull(find["long"])): # if 1st set of coords not available, check next set in next 2 cells
        find["lat"] = row["LatitudeFS"]
        find["long"] = row["LatitudeFS_1"]
        if (pd.isnull(find["lat"]) or pd.isnull(find["long"])): # if still null, use default
            find["lat"] = DEFAULT_LAT
            find["long"] = DEFAULT_LON
            find["cf_custom_region_vague"] = 0
    
    find["find_start_date"] = row["DateFrom"]
    find["find_end_date"] = row["DateTo"]

    find["tempYear"] = row["YearFound"] # temporary, for use in singleOrHoard function
    find["tempCircumstances"] = row["Findcircumstances"] # Same as above

    try:get_date(find, row["YearFound"])
    except: # when YearFound is "" or "no data"
        find["year_found"] = "1700"
        find["year_found_end"] = CURRENT_YEAR
    
    find["comments"] = row["Findcircumstances"]
    find["imported"] = TODAY
    find["references"] = row["Bibliography"]
    find["owner"] = "FRC-PL"
    find["created"] = TODAY
    finds.append(find)
singleOrHoard(freq, finds)

Map each id to its index

In [1177]:
index = 1
for find in finds:
    find_id_to_Index[find["FindNumber"]] = index
    index += 1

Combine duplicate single coin finds to save space

In [1178]:
representatives = defaultdict(list)
toRemove = list()
for find in finds:
    if (find["type_find"] == "Single Find"):
        values =  (
                  find["Name"], find["lat"], find["long"], find["cf_custom_region_vague"], find["find_start_date"], 
                  find["find_end_date"], find["year_found"], find["year_found_end"]
                  )
        rep = representatives.get(values)
        if (rep == None): # coin not found yet, so it now represents any future coins that have the same values
            representatives[values] = find
        else: # coin already has a representative
            rep["num_coins"] += 1
            toRemove.append(find)
for duplicate in toRemove:
    finds.remove(duplicate)

## Groups Processor

In [1179]:
DEFAULT_MINT = "Unknown (Roman Empire)"

In [1180]:
def convertMetal(symbol): # convert metal symbol
    metal_conversion = {
        'Ae': 'bronze', 
        'Ag': 'silver', 
        'Au': 'gold'
    }
    return metal_conversion.get(symbol, "Unknown")

In [1181]:
def convertDenomination(denomination): # Taken from Mark's AFE code
    denomination_conversion = {
    'Denarius':'denarius', 'Semis':'semissis', 'AE1': 'AE1 (8.05g)',
        'AE2 / AE3': 'AE2 (5.15g) or AE3 (2.58g)', 'AE3 / AE2': 'AE3 (2.58g) or AE2 (5.15g)',
            'AE3 / AE4': 'AE3 (2.58g) or AE4 (1.23g)', 'AE4 / AE3': 'AE4 (1.23g) or AE3 (2.58g)',
                'Aureus':'aureus', 'AE':'uncertain (bronze)', 'Antoninianus':'radiate or nummus (UK find)',
                    'Follis':'follis','AE2':'AE2 (5.15g)', 'Drachme':'drachm', 'AV':'AV', 'Maiorina':'follis',
                'AE3':'AE3 (2.58g)', 'Solidus':'solidus', 'Silber':'uncertain (silver)', 'Siliqua':'siliqua', 
            'AE4':'AE4 (1.23g)', 'Tremissis':'tremissis', '10 Num' : '10 nummi', '2 Solidi' : '2 solidi', 
         'Miliarensis' : 'miliarensis', 'Argenteus': 'argenteus', 'Bronze': 'uncertain (bronze)',
    'Solidus / Tremissis': 'solidus or tremissis', 'Siliqua (reduziert)' : 'reduced siliqua',
    }
    return denomination_conversion.get(denomination, "uncertain")

In [1182]:
def convertMint(mint): # converting mints into FLAME format
    # Thanks for this Mark!
    mint_conversion = {
    'Roma':'Roma', 'Alexandria':'Alexandria ad Aegyptum', 'Uncertain mint':'Unknown', 'Constantinopolis':'Constantinople',
       'Unofficial mint':'Unknown', 'Siscia':'Siscia', 'Thessalonica':'Thessalonika', 
       'Thessalonica / Treveri': 'Thessalonika or Colonia Augusta Treverorum', 'Roma / Treveri': 'Roma or Colonia Augusta Treverorum',
       'Treveri / Siscia': 'Colonia Augusta Treverorum or Siscia', 'Londinium':'Londinium', 
    'Treveri / Constantinopolis': 'Colonia Augusta Treverorum or Constantinople', 'Treveri':'Colonia Augusta Treverorum', 
       'Lugdunum':'Lugdunensium', 'Ticinum':'Ticinum', 'Aquileia':'Aquileia', 'Colonia CAA':'Unknown', 'Antiochia':'Antioch', 
       'Emerita':'Emerita', 'Lycia':'Unknown (East Roman)', 'Cyzicus':'Kyzikos', 'Roma / Lugdunum':'Roma or Lugdunum',
        'Sirmium':'Sirmium', 'Eastern mint':'Unknown (East Roman)', 'Gallia':'Unknown (Gaul)', 'Unidentified mint':'Unknown',
    'Laodicea ad Mare':'Laodicea ad Mare', 'Mediolanum':'Mediolanum', 'Constantinopolis / Mediolanum': 'Constantinople or Mediolanum', 
        'Ravenna':'Ravenna', 'Roma / Tarraco (?)':'Roma or Tarracona', 'Africa':'Unknown (Africa)', 'Hispania':'Unknown (Iberia)', 
        'Colonia Caesaraugusta':'Cesaraugusta', 'Greek East':'Unknown (East Roman)', 'Münzstätte nicht bekannt':'Unknown (Germany)', 
        'Östliche Münzstätte' : 'Unknown (Germany)', 'Sicilia' : 'Sicily', 'Syrien' : 'Unknown (Greater Syria)',
    'Arelate' : 'Arelato', 'Heracleia' : 'Heraclea', 'Nicomedia': 'Nicomedia'
    }
    return mint_conversion.get(mint, DEFAULT_MINT)

Combines duplicates and assigns num_coins

In [1183]:
def getNums(groups):
    for group in groups:
        if (len(duplicates[(group["name"], group["denomination"], group["start_year"], group["end_year"], group["mint"])]) > 1):
            dupes = duplicates[(group["name"], group["denomination"], group["start_year"], group["end_year"], group["mint"])]
            index = 1 # to keep 0th one from being removed
            while (index < len(duplicates[(group["name"], group["denomination"], group["start_year"], group["end_year"], group["mint"])])):
                 groups.remove(dupes[index])
                 index += 1
            group["num_coins"] = len(duplicates[(group["name"], group["denomination"], group["start_year"], group["end_year"], group["mint"])])

Setting up groups dataframe

In [1184]:
groups = list()
duplicates = defaultdict(list)
for index, row in theirData.iterrows():
    group = {}
    group["group_index"] = index + 1
    group["name"] = row["FindspotName"]
    group["frcpl_find_id"] = groupToFind.get(row["ID"], row["ID"]) # if not found, default to row["ID"]
    group["mint"] = convertMint(row["Mint"])

    group["denomination"] = convertDenomination(row["Denomination"])

    group["start_year"] = row["DateFrom"]
    group["end_year"] = row["DateTo"]

    ruler = row["Issuer"]
    if (ruler != ruler): # sometimes nan, leaving blank cell
        group["ruler"] = "Uncertain"
    else:
        group["ruler"] = ruler
    
    group["num_coins"] = 1

    id_of_original_find = group["frcpl_find_id"]
    index_of_original_find = find_id_to_Index[id_of_original_find]
    group["coin_find_index"] = "FRCPL-" + str(index_of_original_find)

    group["created"] = TODAY
    group["imported"] = TODAY
    group["owner"] = "FRCPL"
    group["comments"] = row["Findcircumstances"]
    group["metal"] = convertMetal(row["Material"])

    if (group["denomination"] == "Unknown" or group["denomination"] == "unknown" or group["denomination"] == "uncertain" 
        and group["metal"] != "Unknown"):
        group["denomination"] += " (" + group["metal"] + ")"

    duplicates[(group["name"], group["denomination"], group["start_year"], group["end_year"], group["mint"])].append(group)
    groups.append(group)
getNums(groups) # combines duplicates and assigns num_coins

Add FRCPL-X-X indices

In [1185]:
groups[0]["coin_find_group_index"] = "FRCPL-1-1"
index = 0
group_index = 1
for group in groups:
    if (groups[index-1]["coin_find_index"] == group["coin_find_index"]): # if still part of the same find
        group_index += 1
    else:
        group_index = 1
    group["coin_find_group_index"] = group["coin_find_index"] + "-" + str(group_index)
    index += 1
    

Export finds to CSV

In [1186]:
df = pd.DataFrame(finds)
type = df.pop("type_find") # rearranging two columns
num = df.pop("num_coins")
df.insert(2, "type_find", type)
df.insert(5, "num_coins", num)
df.index += 1
df.to_csv("final_frcpl_finds.csv", encoding='utf16', sep='\t')

Export groups to CSV

In [1187]:
df2 = pd.DataFrame(groups)
find_index = df2.pop("coin_find_index")
find_group_index = df2.pop("coin_find_group_index")
df2.insert(1, "coin_find_index", find_index)
df2.insert(2, "coin_find_group_index", find_group_index)
df2.index += 1
df2.to_csv("final_frcpl_groups.csv", encoding='utf16', sep='\t')